In [1]:
using IFSintegrals

# IMPORTANT NOTICE
This was the original prototype for the code. There are a few bugs in here, which were removed in ```Jacobi_matrices.jl```

# Lemma 1

### Lemma 1 details

From the three-term recurrence relation, we have
$$
r_np_n(x) = \left(x-A_{n-1}\right)p_{n-1}(x) - r_{n-1}p_{n-2}(x),
$$
from which it follows immediately that
$$
r_np_n(\phi_i(x)) = \left(\phi_i(x)-A_{n-1}\right)p_{n-1}(\phi_i(x)) - r_{n-1}p_{n-2}(\phi_i(x)),
$$
and then assuming we have the coefficients for lower $n$'s, we can write
$$
\begin{align}
r_np_n(\phi_i(x)) &= \left((\delta_ix+\beta_i)-A_{n-1}\right)p_{n-1}(\phi_i(x)) - r_{n-1}p_{n-2}(\phi_i(x))\\
 &= \left(\delta_ix+\beta_i-A_{n-1}\right)\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}p_\ell(x) - r_{n-1}\sum_{\ell=0}^{n-2}\Gamma_{i,\ell}^{n-2}p_\ell(x)\\
 &= \left(\beta_i-A_{n-1}\right)\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}p_\ell(x) +\delta_ix\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}p_\ell(x) - r_{n-1}\sum_{\ell=0}^{n-2}\Gamma_{i,\ell}^{n-2}p_\ell(x)
\end{align}
$$
and then taking inner products gives
$$
\begin{align}
\left(r_np_n\circ\phi_i,p_j\right) &= \left(\left(\delta_i-A_{n-1}\right)\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}p_\ell(x) +\delta_ix\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}p_\ell(x) - r_{n-1}\sum_{\ell=0}^{n-2}\Gamma_{i,\ell}^{n-2}p_\ell(x),p_j(x)\right)\\
&=\left(\beta_i-A_{n-1}\right)\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}(p_\ell,p_j) +\delta_i\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}(xp_\ell,p_j) - r_{n-1}\sum_{\ell=0}^{n-2}\Gamma_{i,\ell}^{n-2}(p_\ell,p_j).\\
\end{align}
$$
Now assuming that $j\leq n-2$, we can write
$$
\begin{align}
r_n\left(p_n\circ\phi_i,p_j\right) =\left(\beta_i-A_{n-1}\right)\Gamma_{i,j}^{n-1} +\delta_i\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}(p_\ell(x)x,p_j(x)) - r_{n-1}\Gamma_{i,j}^{n-2}.
\end{align}
$$
Now focus on $(xp_\ell,p_j)$. There are four cases, which follow by orthogonality:
$$
(xp_\ell,p_j) = \gamma_{\ell,j}:=\left\{\begin{array}{cc}
r_j=r_\ell & j=\ell\\
r_\ell & \ell=j-1\\
r_j & j=\ell-1\\
0 & \text{otherwise}
\end{array}\right.
$$
The cases for $j=n-1$ follows similarly, removing terms as required. For $j=n$, we have
$$
r_n\left(p_n\circ\phi_i,p_n\right) = \delta_i\Gamma_{i,n-1}^{n-1}(xp_{n-1},p_n)
$$

we have $r_n\left(p_n\circ\phi_i,p_n\right) = \delta_i\Gamma^{n-1}_{i,n-1}r_n$, hence
$$
\left(p_n\circ\phi_i,p_n\right) = \delta_i\Gamma^{n-1}_{i,n-1}
$$
perhaps this is what is meant by _in passage_ in the proof of Lemma 1. But the $r_n$ term is needed for lower $j$, as we must divide through by it, and it won't cancel so neatly.

### Lemma 1 spin off
Immediately after Lemma 1, there is the statement that an alternative recursive process would lead to
$$
\tilde{p}_n(\phi_i(x)) = \tilde{\Gamma}_{i,n}^n\tilde{p}_n(x) + \sum_{\ell=0}^{n-1}\tilde{\Gamma}_{i,\ell}p_{\ell}(x),
$$
where $\tilde p_n := r_np_n$.

In the above, we could equivalently write
$$
\left(\tilde{p}_n\circ\phi_i,\tilde{p}_n\right) = \delta_i\Gamma_{i,n-1}^{n-1}\frac{(xp_{n-1},\tilde{p}_n)}{(\tilde{p}_n,\tilde{p}_n)} = \delta_i\Gamma^{n-1}_{i,n-1},
$$
and
$$
\begin{align}
\left(\tilde{p}_n\circ\phi_i,p_j\right) =\left(\delta_i-A_{n-1}\right)\Gamma_{i,j}^{n-1} +\delta_i\sum_{\ell=0}^{n-1}\Gamma_{i,\ell}^{n-1}(p_\ell(x)x,p_j(x)) - r_{n-1}\Gamma_{i,j}^{n-2},
\end{align}
$$
so the process in the above block can be used to determine coefficients which don't involve $r_n$. It would have been much simpler to write
$\tilde\Gamma_{in}^n:=\Gamma_{in}^n$ and $\tilde{\Gamma}_{i\ell}^n:={r_n}{\Gamma_{i\ell}^n}$ for $\ell=0,\ldots,n-1$.

In [2]:
#Now let's try and implement the above#
function get_modified_coeffs_from_coeffs(coeffs_one_below::Vector{Float64},coeffs_two_below::Vector{Float64},A::Vector{Float64},r::Vector{Float64},sₘ::Similarity)
   # rule of thumb: square bracket indices should always be incremented by one compared with what's written above,
    # to account for the zero index
    
    n = length(coeffs_one_below) # one more because of the zeroth entry, one less because we're incrementing
    modified_coeffs = zeros(Float64,n+1)
    
    function γ(ℓ::Integer,j::Integer)
        if ℓ==j
            return A[ℓ+1]*coeffs_one_below[j]
        elseif ℓ==j-1
            return r[ℓ+1]*coeffs_one_below[j]
        elseif j==ℓ-1
            return r[j+1]*coeffs_one_below[j]
        else
            return 0.0
        end
    end
        
    function get_trickier_sum(j)
        trickier_sum = 0.0
        for ℓ=max(0,j-1):min(n-1,j+1)
            trickier_sum += γ(ℓ,j)
        end
        return trickier_sum
    end
        
    for j = 0:(n-2)
        modified_coeffs[j+1] = (sₘ.δ-A[n])*coeffs_one_below[j] + sₘ.r*get_trickier_sum(j+1) - r[n]*coeffs_two_below[j]
    end
    
    # now deal with j = n-1
    modified_coeffs[n] = (sₘ.δ-A[n])*coeffs_one_below[n] + sₘ.r*get_trickier_sum(n)
    
    # now deal with j = n, which is not in orthanormal function
    modified_coeffs[n+1] = sₘ.r*coeffs_one_below[n]
    
    return modified_coeffs
end

get_modified_coeffs_from_coeffs (generic function with 1 method)

Syntax reminder:
```get_modified_coeffs_from_coeffs(coeffs_one_below::Vector{Float64},coeffs_two_below::Vector{Float64},A::Vector{Float64},r::Vector{Float64},sₘ::Similarity)```

Now let's try it out with some randomish data:

In [3]:
φᵢ = Similarity(1/3,0.4)
A = [1.0]
r = [0.0]
Γ⁰ = [1.0]

1-element Vector{Float64}:
 1.0

In [4]:
get_modified_coeffs_from_coeffs(Γ⁰, Γ⁰, A, r, φᵢ)

2-element Vector{Float64}:
 -0.6
  0.3333333333333333

# Lemma 2

There is a hidden step here, but fortunately more similar to what we've seen before. Rearranging (27) gives

$$
\begin{align}
r_n^2-\sum_{i=1}^M\pi_iD_i &= \sum_{i=1}^M\pi_i(B_i+C_i)\\
r_n^2-r_n^2\sum_{i=1}^M\pi_i\delta_i\tilde{\Gamma}^n_{i,n}\Gamma^{n-1}_{i,n-1} &= \sum_{i=1}^M\pi_i(B_i+C_i)\\
r_n^2\left(1-\sum_{i=1}^M\pi_i\delta_i\tilde{\Gamma}^n_{i,n}\Gamma^{n-1}_{i,n-1}\right) &= \sum_{i=1}^M\pi_i(B_i+C_i)\\
r_n &=\sqrt{\frac{\sum_{i=1}^M\pi_i(B_i+C_i)}{1-\sum_{i=1}^M\pi_i\delta_i\tilde{\Gamma}^n_{i,n}\Gamma^{n-1}_{i,n-1}}}\\
r_n &=\sqrt{\frac{\sum_{i=1}^M\pi_i(B_i+C_i)}{1-\sum_{i=1}^M\pi_i\tilde{D}_i}}
\end{align}
$$
where $\tilde{D}_i:= \delta_i\tilde{\Gamma}^n_{i,n}\Gamma^{n-1}_{i,n-1}$

In [16]:
function get_rₙ(modified_coeffs::Vector{Vector{Float64}}, coeffs_one_below::Vector{Vector{Float64}}, A::Vector{Float64}, Γ::SelfSimilarFractal)
   n = length(modified_coeffs::Vector{Vector{Float64}})+1
    M = length(Γ.IFS)
    B = zeros(M)
    C = zeros(M)
    D = zeros(M)
    
    for i=1:M
        for ℓ=0:(n-1)
           B[i] += (Γ.IFS[i].δ + Γ.IFS[i].r*A[ℓ+1])*modified_coeffs[i][ℓ+1]*coeffs_one_below[i][ℓ+1]
        end
        for ℓ=0:(n-2)
           C[i] +=  Γ.IFS[i].r*r[ℓ+2]*(modified_coeffs[i][ℓ+1]*coeffs_one_below[i][ℓ+2] + modified_coeffs[i][ℓ+2]*coeffs_one_below[i][ℓ+1])
        end
        D_nominator[m] = Γ.IFS[i].r*modified_coeffs[i][n+1]*coeffs_one_below[i][ℓ+1]
    end
    
    rₙ² = (Γ.weights'*(B+C)) / (1-(Γ.weights'*D_nominator))
    
    return sqrt(rₙ²)
end

get_rₙ (generic function with 1 method)

# Lemma 3
As with Lemma 2, a tiny bit of rearranging is necessary, from (32):
$$
\begin{align}
    A_n &= \sum_{i=1}^M\pi_i\left[\sum_{m=0}^n(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \sum_{m=0}^{n-1}\Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right]\\
        A_n &= \sum_{i=1}^M\pi_i\left[\sum_{m=0}^{n-1}(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \sum_{m=0}^{n-1}\Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right] + \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2(\beta_i+\delta_iA_n)\\
        &= \sum_{i=1}^M\pi_i\left[\sum_{m=0}^{n-1}(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \sum_{m=0}^{n-1}\Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right] + \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\delta_iA_n + \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\beta_i\\
         A_n - A_n\sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\delta_i &=\sum_{i=1}^M\pi_i\left[\sum_{m=0}^{n-1}(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \sum_{m=0}^{n-1}\Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right] + \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\beta_i\\
         A_n\left(1 - \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\delta_i\right) &=\sum_{i=1}^M\pi_i\left[\sum_{m=0}^{n-1}(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \sum_{m=0}^{n-1}\Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right] + \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\beta_i\\
         A_n &=\left(1 - \sum_{i=1}^M\pi_i(\Gamma_{i,n}^n)^2\delta_i\right)^{-1}\left(\sum_{i=1}^M\pi_i\left[(\Gamma_{i,n}^n)^2\beta_i+\sum_{m=0}^{n-1}(\Gamma_{i,m}^n)^2(\beta_i+\delta_iA_m) + \Gamma^{n}_{i,m}\Gamma^n_{i,m+1}\delta_i(r_m+r_{m+1})\right] \right)
\end{align}
$$

In [6]:
function get_Aₙ(coeffs_this_level::Vector{Vector{Float64}}, A::Vector{Float64}, r::Vector{Float64}, Γ::SelfSimilarFractal)
    
    n = length(coeffs_this_level)+1
    big_sum = 0.0
    denominator = 1.0
    for i=1:M
        big_sum += coeffs_this_level[n+1]^2*IFS[i].δ
        for m=0:(n-1)
            big_sum += coeffs_this_level[m+1]^2*(IFS[i].δ+IFS[i].r*A[m+1]) + coeffs_this_level[m+1]*coeffs_this_level[m+2]*IFS[m].r*(r[m+1]+r[m+2])
        end
        big_sum *= Γ.weights[i]
        denominator += Γ.weights[i]*coeffs_this_level[m+1]^2*Γ.IFS[i].r
    end
    return big_sum/(1-denominator)
end

get_Aₙ (generic function with 1 method)

# Now attempt to glue this shitshow together.

Need to restrict to fractals $\Gamma\subset\mathbb{R}$.

In [12]:
function get_Jacobi_matrix(Γ::Attractor,N::Int64)
    # initialisation
   A = [Γ.measure*Γ.barycentre] # checks out, given def'n of barycentre, should be ∫_Γ x dμ(x)
    r = [0.0]
    M = length(Γ.IFS)
    coeffs_one_below = [[1.0] for _=1:M]
    coeffs_two_below = [[0.0] for _=1:M]
    
    # iteration
    for n=1:N #we've done n=0 above
        # initialise this level
        coeffs_this_level = [zeros(n+1) for _=1:M]
        
        # step one
        modified_coeffs = [zeros(n+1) for _=1:M]
        for m=1:M
            modified_coeffs[m] = get_modified_coeffs_from_coeffs(coeffs_one_below[m], coeffs_two_below[m], A, r, Γ.IFS[m])
        end
        
        # step two
        rₙ = get_rₙ(modified_coeffs, coeffs_one_below, A, Γ)
        push!(r,rₙ)
        
        # step three
        for m=1:M
            coeffs_this_level[m][1:n] = modified_coeffs[m][1:n]/r[n+1]
            coeffs_this_level[m][n+1] = modified_coeffs[m][n+1]
        end
        
        # step four
        Aₙ = get_Aₙ(coeffs_this_level, A, r, Γ)
        push!(A,Aₙ)
        
        coeffs_two_below = coeffs_one_below
        coeffs_one_below = coeffs_this_level
        
    end
    
    # now make the Jacobi matrix
    J = zeros(Float64,N,N)
    for i=0:(N-1)
       J[i+1,i+1] = A[i+1]
        J[i+1,i] = r[i+2]
        J[i,i+1] = r[i+2]
    end
    J[N,N] = A[N+1]
    
    return J
end

get_Jacobi_matrix (generic function with 2 methods)

In [17]:
Γ = CantorSet()
get_Jacobi_matrix(Γ,2)

LoadError: BoundsError: attempt to access 1-element Vector{Float64} at index [2]